In [234]:
import pandas as pd
import requests
import geopandas as gpd
import ast
import os
import numpy as np
import time
import json

In [2]:
#Reading the suburb data
suburb = pd.read_csv('Data/plants_suburbs.csv')
suburb.head()

,sci_name,suburb
0,Pomaderris cotoneaster,['GOONGERAH']
1,Caladenia tessellata,"['DARGO', 'KALIMNA WEST', 'MACKS CREEK', 'CLOV..."
2,Taraxacum cygnorum,"['KOROIT', 'DRUMBORG', 'MUMBANNAR']"
3,Acacia phasmoides,['PINE MOUNTAIN']
4,Asterolasia phebalioides,"['VICTORIA VALLEY', 'GLENISLA', 'BELLFIELD', '..."


In [6]:
#reading the shape file to get the lat and long for the suburb
shape_file = gpd.read_file('vic_locality_polygon_shp/VIC_LOCALITY_POLYGON_shp.shp')
shape_file.head()

,LC_PLY_PID,DT_CREATE,DT_RETIRE,LOC_PID,VIC_LOCALI,VIC_LOCA_1,VIC_LOCA_2,VIC_LOCA_3,VIC_LOCA_4,VIC_LOCA_5,VIC_LOCA_6,VIC_LOCA_7,geometry
0,6670,2011-08-31,None,VIC2615,2012-04-27,None,UNDERBOOL,None,None,G,None,2,"POLYGON ((141.74552 -35.07229, 141.74552 -35.0..."
1,6671,2011-08-31,None,VIC1986,2012-04-27,None,NURRAN,None,None,G,None,2,"POLYGON ((148.66877 -37.39571, 148.66876 -37.3..."
2,6672,2011-08-31,None,VIC2862,2012-04-27,None,WOORNDOO,None,None,G,None,2,"POLYGON ((142.92288 -37.97886, 142.90449 -37.9..."
3,6673,2011-08-31,None,VIC734,2018-08-03,None,DEPTFORD,None,None,G,None,2,"POLYGON ((147.82336 -37.66001, 147.82313 -37.6..."
4,6674,2011-08-31,None,VIC2900,2012-04-27,None,YANAC,None,None,G,None,2,"POLYGON ((141.27978 -35.99859, 141.27989 -35.9..."


In [11]:
ast.literal_eval(suburb.suburb[0])

['GOONGERAH']

In [13]:
all_suburbs = []
for index,value in suburb.iterrows():
    sub_list = ast.literal_eval(suburb['suburb'][index])
    for i in sub_list:
        all_suburbs.append(i)
        

In [18]:
all_suburbs = list(set(all_suburbs))

In [20]:
#Creating the dataset for all the suburbs
new_df = pd.DataFrame(columns = ['suburb','polygon','lat','long'])

In [24]:
#subset the shape file
shape_file = shape_file[['VIC_LOCA_2','geometry']].rename(columns={'VIC_LOCA_2':'suburb'})

In [25]:
shape_file['location'] = shape_file['geometry'].centroid

In [26]:
shape_file

,suburb,geometry,location
0,UNDERBOOL,"POLYGON ((141.74552 -35.07229, 141.74552 -35.0...",POINT (141.82492 -35.15327)
1,NURRAN,"POLYGON ((148.66877 -37.39571, 148.66876 -37.3...",POINT (148.58121 -37.49585)
2,WOORNDOO,"POLYGON ((142.92288 -37.97886, 142.90449 -37.9...",POINT (142.81667 -37.87948)
3,DEPTFORD,"POLYGON ((147.82336 -37.66001, 147.82313 -37.6...",POINT (147.68843 -37.56620)
4,YANAC,"POLYGON ((141.27978 -35.99859, 141.27989 -35.9...",POINT (141.40838 -36.08582)
...,...,...,...
2968,FRASER RISE,"POLYGON ((144.73252 -37.71121, 144.73252 -37.7...",POINT (144.71262 -37.70482)
2969,CAROLINE SPRINGS,"POLYGON ((144.74615 -37.74659, 144.74610 -37.7...",POINT (144.73721 -37.73447)
2970,SOMERS,"POLYGON ((145.19211 -38.39105, 145.19392 -38.3...",POINT (145.16097 -38.38140)
2971,HMAS CERBERUS,"POLYGON ((145.21831 -38.38722, 145.21863 -38.3...",POINT (145.19831 -38.38123)


In [105]:
 vic_suburbs_plants = shape_file[shape_file['suburb'].isin(all_suburbs)]

### Plan to get the weather data.
1. Get all the weather stations with station'id's and names in victoria.
2. Iterate through all the csv files to get the climate data based on the month.

In [32]:
#Getting all the stations in victoria
#read the stations data over australia
stations = pd.read_csv('bometa20150501-20160430/BoM_ETA_20150501-20160430/spatial/StationData.csv')
stations.head()

,WMO_NUM,station_number,station_name,LATITUDE,LONGITUDE,STN_HT,AVIATION_ID,REGION,GridPt Lat,GridPt Lon,MSAS elevation,Distance from GridPt,Roughness,Distance from coast,Category,forecast_district,sa_special
0,95695,46012,WILCANNIA AERODROME AWS,-31.5194,143.3850,94.3,YWCA,NSW,-31.49,143.38,89.12,2.4,13.3,999.0,flat_inland,NSW_PW016,NaN
1,95485,46126,TIBOOBURRA AIRPORT,-29.4448,142.0567,176.4,YTIB,NSW,-29.45,142.04,170.04,2.1,11.7,999.0,flat_inland,NSW_PW016,NaN
2,94686,46128,FOWLERS GAP AWS,-31.0867,141.7017,181.0,FOWG,NSW,-31.07,141.71,190.08,0.7,21.0,225.0,flat_inland,NSW_PW016,NaN
3,95699,46129,WHITE CLIFFS AWS,-30.8517,143.0742,158.0,YWHC,NSW,-30.82,143.08,150.88,2.2,8.4,999.0,flat_inland,NSW_PW016,NaN
4,94691,47048,BROKEN HILL AIRPORT AWS,-32.0012,141.4694,281.3,YBHI,NSW,-31.99,141.46,289.28,1.1,15.3,199.0,flat_inland,NSW_PW015,NaN


In [35]:
vic_stations = stations[stations.REGION =='VIC'].reset_index().drop(['index'],axis=1)

In [36]:
test_sample = pd.read_csv('bometa20150501-20160430/BoM_ETA_20150501-20160430/obs/obs_hourly_20150501.csv')
test_sample.head()

,station_number,area_code,parameter,valid_start,valid_end,value,unit,statistic,instantaneous,level,qc_valid_minutes,qc_valid_minutes_start,qc_valid_minutes_end
0,1006,WA_PT174,AIR_TEMP,1430409600,1430409600,19.8,Celsius,first,True,SFC,NaN,1430409600,NaN
1,1006,WA_PT174,AIR_TEMP,1430413200,1430413200,18.7,Celsius,first,True,SFC,NaN,1430413200,NaN
2,1006,WA_PT174,AIR_TEMP,1430416800,1430416800,17.3,Celsius,first,True,SFC,NaN,1430416800,NaN
3,1006,WA_PT174,AIR_TEMP,1430420400,1430420400,16.3,Celsius,first,True,SFC,NaN,1430420400,NaN
4,1006,WA_PT174,AIR_TEMP,1430424000,1430424000,14.7,Celsius,first,True,SFC,NaN,1430424000,NaN


In [39]:
all_weather_files = []
directory = os.path.join("bometa20150501-20160430/BoM_ETA_20150501-20160430/obs/")
for root,dirs,files in os.walk(directory):
    for file in files:
        if file.endswith(".csv"):
            #f=open(file, 'r')
            #f.close()
            all_weather_files.append(file)

In [49]:
check_value = pd.DataFrame(test_sample.groupby('station_number')['value'].mean()).reset_index()

In [59]:
stationnumbers = list(vic_stations.station_number)


,WMO_NUM,station_number,station_name,LATITUDE,LONGITUDE,STN_HT,AVIATION_ID,REGION,GridPt Lat,GridPt Lon,MSAS elevation,Distance from GridPt,Roughness,Distance from coast,Category,forecast_district,sa_special
0,94693,76031,MILDURA AIRPORT,-34.2358,142.0867,50.0,YMIA,VIC,-34.24,142.08,44.00,1.6,4.5,999.0,flat_inland,VIC_PW001,NaN
1,95831,76064,WALPEUP RESEARCH,-35.1201,142.0040,105.0,WALP,VIC,-35.12,142.00,94.60,0.7,9.5,999.0,flat_inland,VIC_PW001,NaN
2,94838,77010,HOPETOUN AIRPORT,-35.7151,142.3569,77.3,YHPN,VIC,-35.70,142.38,78.76,1.8,2.6,999.0,flat_inland,VIC_PW002,NaN
3,94843,77094,SWAN HILL AERODROME,-35.3766,143.5416,71.0,YSWH,VIC,-35.37,143.54,72.68,0.2,6.2,999.0,flat_inland,VIC_PW001,NaN
4,94827,78015,NHILL AERODROME,-36.3092,141.6486,138.9,YNHL,VIC,-36.28,141.67,147.64,2.5,8.6,197.0,flat_inland,VIC_PW002,NaN


In [48]:
all_weather_files[1:3]

['obs_hourly_20150502.csv', 'obs_hourly_20150503.csv']

In [63]:
all_weather_files[0].split('_')[2]

'20150501.csv'

In [62]:
all_weather_files[0].split('_')[2][4:8]

'0501'

In [86]:
stationnumbers = list(vic_stations.station_number)
#Dictonary to store the temperatures at month level
month_dict = {}
#dictionary to store the temperaures at day level.
day_dict = {}
#iterating over all the csv file of the temperature
for file in all_weather_files:
    temp_file = pd.read_csv('bometa20150501-20160430/BoM_ETA_20150501-20160430/obs/'+file)
    #group the dataframe as they are recorded for every hour
    grouping = pd.DataFrame(temp_file.groupby('station_number')['value'].mean()).reset_index()
    #consider only the stations which are in victoria
    filter1 = grouping[grouping['station_number'].isin(stationnumbers)]
    #store the resulting file in a day_dict dictonary with month and day as the key.
    day_dict[file.split('_')[2][4:8]] = filter1

In [87]:
months = ['05','06','07','08','09','10','11','12','01','02','03','04']
baselist = []
#iterating over the months
for number in months:
    #iterating over the keys of the day_dict dictionary
    for value in list(day_dict.keys()):
        #if the key is of particular month from the above loop store the dataframe
        if value.startswith(number):
            baselist.append(day_dict[value])
    #finding the mean of all the data for the whole month
    monthfilter = pd.DataFrame(pd.concat(baselist,axis=0).groupby('station_number')['value'].mean()).reset_index()
    #store the data in another dataframe
    month_dict[number] = monthfilter
    #clear the temp list
    baselist = []
    


In [90]:
month_convert = {'05':'May','06':'June','07':'July','08':'August',
                '09':'September','10':'October','11':'November','12':'December',
                '01':'January','02':'February','03':'March','04':'April'}

In [97]:
all_temp = pd.DataFrame(columns=['station_number'])
all_temp['station_number'] = stationnumbers
for value in list(month_dict.keys()):
    temp_df = month_dict[value]
    temp_df = temp_df.rename(columns={'value':month_convert[value]})
    merged = pd.merge(all_temp,temp_df,on='station_number',how='inner')
    all_temp = merged

In [98]:
all_temp

,station_number,May,June,July,August,September,October,November,December,January,February,March,April
0,76031,9.874429,7.336771,6.721606,8.310249,10.179861,16.205477,16.664201,19.754469,19.297062,18.947773,18.149026,13.656111
1,76064,9.751949,7.477153,6.647849,7.673185,9.790660,15.677587,15.734167,19.212668,18.763389,18.065158,17.470262,13.625000
2,77010,9.043515,6.927708,6.069288,7.307258,9.000208,14.822984,15.113264,18.614247,18.288743,17.457256,16.955645,12.952639
3,77094,9.170363,6.904201,5.934543,7.400672,9.012326,15.102050,15.734375,18.860215,18.822098,18.201904,17.605880,13.132257
4,78015,8.755693,6.840971,6.129690,6.971725,8.612387,13.967873,13.830312,17.644220,17.134100,16.064116,15.590558,12.356667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,90184,9.587399,8.722743,7.650470,8.016431,9.099479,11.148085,11.155799,12.870296,13.217835,13.442960,12.923723,12.016354
69,90186,8.453293,7.452604,6.279326,6.812401,7.920440,10.902218,10.978264,13.289987,13.696415,13.238757,12.807198,11.180694
70,90194,8.309577,7.494132,6.318347,6.848690,8.110660,11.572782,11.461042,14.392204,14.422212,13.716739,13.049261,11.033924
71,200838,9.628461,8.300660,7.551008,7.371243,8.512512,10.755331,10.856522,12.610047,13.315465,13.822414,13.355276,12.085742


# Getting the soil type

In [126]:
lat = list(vic_suburbs_plants['location'])[0].x
long = list(vic_suburbs_plants['location'])[0].y

In [127]:
import requests

In [139]:
response = requests.get('https://api.vic.gov.au:443/avr/soils-api/v1.0/estimates?wgs84_east=148.58121156267907&wgs84_north=-37.49585026204892',
                       headers = {
  "Content-Type": "application/json",
  "Accept": "application/json",
  "apikey": "908247fe-e800-42a0-937f-7e7b56fe8372"
})

In [155]:
vic_suburbs_plants['soil_type'] = np.nan

C:\Users\Chandra\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [174]:
count = 0
for index,value in vic_suburbs_plants.iterrows():
    lat = vic_suburbs_plants.location[index].x
    long = vic_suburbs_plants.location[index].y
    url = f'https://api.vic.gov.au:443/avr/soils-api/v1.0/estimates?wgs84_east={lat}&wgs84_north={long}'
    response = requests.get(url, headers = {
  "Content-Type": "application/json",
  "Accept": "application/json",
  "apikey": "908247fe-e800-42a0-937f-7e7b56fe8372"
})
    #store the soil type
    dic = json.loads(response.content)
    try:
        vic_suburbs_plants['soil_type'][index]=dic['estimates']['soil_texture_topsoil']['text']
    except KeyError as e:
        vic_suburbs_plants['soil_type'][index] = 'Loam'
    count = count +1
    if count==20:
        time.sleep(60)
        count = 0

C:\Users\Chandra\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Chandra\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3319: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\Chandra\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [216]:
suburb['soil_type']='Soil type'
for index,value in suburb.iterrows():
    #get the list of suburbs
    mid_suburbs = suburb['suburb'][index]
    #print(mid_suburbs)
    mid_suburbs = ast.literal_eval(mid_suburbs)
#get soil names for all the suburbs for that particualr plant  
    mid_soil = []
    for i in mid_suburbs:
        get_soil = vic_suburbs_plants[vic_suburbs_plants['suburb']==i]['soil_type']
    
        mid_soil.append((get_soil))
    
    suburb['soil_type'][index] = mid_soil
    

In [217]:
suburb

,sci_name,suburb,soil_type
0,Pomaderris cotoneaster,['GOONGERAH'],[[Sandy Loam]]
1,Caladenia tessellata,"['DARGO', 'KALIMNA WEST', 'MACKS CREEK', 'CLOV...","[[Sandy Loam], [Sandy Loam], [Fine Sandy Clay ..."
2,Taraxacum cygnorum,"['KOROIT', 'DRUMBORG', 'MUMBANNAR']","[[Clay Loam], [Loamy Sand], [Loamy Sand]]"
3,Acacia phasmoides,['PINE MOUNTAIN'],[[Sandy Loam]]
4,Asterolasia phebalioides,"['VICTORIA VALLEY', 'GLENISLA', 'BELLFIELD', '...","[[Sandy Loam], [Loamy Sand], [Loamy Sand, Loam..."
...,...,...,...
139,Caladenia orientalis,"['TARWIN LOWER', 'WILSONS PROMONTORY', 'WONTHA...","[[Loam], [Sandy Loam], [Loam], [Sand], [Loamy ..."
140,Pelargonium sp. Striatellum (G.W.Carr 10345),['BENAMBRA'],[[Sandy Loam]]
141,Commersonia prostrata,"['LAKES ENTRANCE', 'LAKE WELLINGTON', 'WOODSID...","[[Sandy Loam], [Loam], [Loam], [Sandy Loam], [..."
142,Spyridium furculentum,"['NURCOUNG', 'GYMBOWEN', 'MITRE', 'GRASS FLAT']","[[Loamy Sand], [Sandy Loam], [Sandy Loam], [Sa..."


In [223]:
suburb['soil_type']=suburb['soil_type'].apply(lambda x: [item for elem in x for item in elem])

In [228]:
suburb['soil_type'] = suburb['soil_type'].apply(lambda x: list(set(x)))

In [230]:
#output the soils data for each file.
suburb.to_csv('soils.csv',index=False)

In [238]:
suburb

,sci_name,suburb,soil_type
0,Pomaderris cotoneaster,['GOONGERAH'],[Sandy Loam]
1,Caladenia tessellata,"['DARGO', 'KALIMNA WEST', 'MACKS CREEK', 'CLOV...","[Sandy Loam, Fine Sandy Clay Loam, Loam]"
2,Taraxacum cygnorum,"['KOROIT', 'DRUMBORG', 'MUMBANNAR']","[Loamy Sand, Clay Loam]"
3,Acacia phasmoides,['PINE MOUNTAIN'],[Sandy Loam]
4,Asterolasia phebalioides,"['VICTORIA VALLEY', 'GLENISLA', 'BELLFIELD', '...","[Loam, Loamy Sand, Light Clay, Medium Clay, Li..."
...,...,...,...
139,Caladenia orientalis,"['TARWIN LOWER', 'WILSONS PROMONTORY', 'WONTHA...","[Sand, Loam, Loamy Sand, Light Medium Clay, Cl..."
140,Pelargonium sp. Striatellum (G.W.Carr 10345),['BENAMBRA'],[Sandy Loam]
141,Commersonia prostrata,"['LAKES ENTRANCE', 'LAKE WELLINGTON', 'WOODSID...","[Sandy Loam, Clay Loam, Sand, Loam]"
142,Spyridium furculentum,"['NURCOUNG', 'GYMBOWEN', 'MITRE', 'GRASS FLAT']","[Sandy Loam, Loamy Sand]"


## Categorizing the plants
1. Outdoor Decorative plants
2. Indoor Decorative plants
3. Medicinal Plants
4. Fragrant plants.

In [231]:
cat_plants = pd.read_csv('category.csv')

In [242]:
cat_plants['category'].value_counts()

Decorative    39
Inhouse       10
Fragrant       4
Medicinal      2
Insects        1
Outdoor        1
Name: category, dtype: int64

In [247]:
 decorative_outdoor = cat_plants[cat_plants['category']=='Decorative'].drop(['Common Name','Decorative','Inhouse','Medicinal','Insects','Fragrant'],axis=1).reset_index().drop(['index'],axis=1)

In [249]:
#Getting the inhouse plants
Indoor_plantts = cat_plants[cat_plants['category']=='Inhouse'].drop(['Common Name','Decorative','Inhouse','Medicinal','Insects','Fragrant'],axis=1).reset_index().drop(['index'],axis=1)

In [251]:
#Getting the fragrant plants
fragrance_plants = cat_plants[cat_plants['category']=='Fragrant'].drop(['Common Name','Decorative','Inhouse','Medicinal','Insects'],axis=1).reset_index().drop(['index'],axis=1)

In [252]:
#Output all the csv files.
decorative_outdoor.to_csv('Garden_decorative.csv',index=False)
Indoor_plantts.to_csv('Indoor_decorative.csv',index=False)
fragrance_plants.to_csv('Fragrance_plants.csv',index=False)

In [254]:
fragrance_plants

,sci_name,Threatened status,category,Fragrant
0,Caladenia concolor,Vulnerable,Fragrant,mandarin-flavoured smell
1,Diuris fragrantissima,Endangered,Fragrant,smelling agreeably
2,Zieria citriodora,Vulnerable,Fragrant,Lemon-scented
3,Prasophyllum suaveolens,Endangered,Fragrant,Lavender-lemon


In [255]:
suburb

,sci_name,suburb,soil_type
0,Pomaderris cotoneaster,['GOONGERAH'],[Sandy Loam]
1,Caladenia tessellata,"['DARGO', 'KALIMNA WEST', 'MACKS CREEK', 'CLOV...","[Sandy Loam, Fine Sandy Clay Loam, Loam]"
2,Taraxacum cygnorum,"['KOROIT', 'DRUMBORG', 'MUMBANNAR']","[Loamy Sand, Clay Loam]"
3,Acacia phasmoides,['PINE MOUNTAIN'],[Sandy Loam]
4,Asterolasia phebalioides,"['VICTORIA VALLEY', 'GLENISLA', 'BELLFIELD', '...","[Loam, Loamy Sand, Light Clay, Medium Clay, Li..."
...,...,...,...
139,Caladenia orientalis,"['TARWIN LOWER', 'WILSONS PROMONTORY', 'WONTHA...","[Sand, Loam, Loamy Sand, Light Medium Clay, Cl..."
140,Pelargonium sp. Striatellum (G.W.Carr 10345),['BENAMBRA'],[Sandy Loam]
141,Commersonia prostrata,"['LAKES ENTRANCE', 'LAKE WELLINGTON', 'WOODSID...","[Sandy Loam, Clay Loam, Sand, Loam]"
142,Spyridium furculentum,"['NURCOUNG', 'GYMBOWEN', 'MITRE', 'GRASS FLAT']","[Sandy Loam, Loamy Sand]"


In [257]:
threat_platns = pd.read_csv('Data/endangered_plants.csv')
threat_platns.columns

Index(['sci_name', 'Common Name', 'Threatened status', 'Profile'], dtype='object')

In [260]:
threat_platns['Threatened status'].value_counts()

Vulnerable               86
Endangered               48
Critically Endangered    10
Name: Threatened status, dtype: int64

In [265]:
fragrance_plants = cat_plants[cat_plants['category']=='Medicinal'].drop(['Common Name','Decorative','Inhouse','Insects','Fragrant'],axis=1).reset_index().drop(['index'],axis=1)

In [267]:
fragrance_plants.to_csv('medicinal_plants.csv',index=False)